In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
from tqdm import tqdm
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
# ============================================================
# 1. Dataset Preparation
# ============================================================

class StyleDataset(Dataset):
    def __init__(self, df, tokenizer, label2id, max_len=384, model_type='cross'):
        self.df = df.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.label2id = label2id
        self.max_len = max_len
        self.model_type = model_type

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        orig = row["ORIGINAL TRANSCRIPTS"]
        trans = row["CHANGED STYLE"]
        label = self.label2id[row["PROMPT STYLE"]]

        if self.model_type == 'cross':
            enc = self.tokenizer(orig, trans, truncation=True, padding='max_length', max_length=self.max_len, return_tensors='pt')
            return {
                "input_ids": enc["input_ids"].squeeze(0),
                "attention_mask": enc["attention_mask"].squeeze(0),
                "label": torch.tensor(label, dtype=torch.long)
            }
        else:
            enc_orig = self.tokenizer(orig, truncation=True, padding='max_length', max_length=self.max_len, return_tensors='pt')
            enc_trans = self.tokenizer(trans, truncation=True, padding='max_length', max_length=self.max_len, return_tensors='pt')
            return {
                "orig_input_ids": enc_orig["input_ids"].squeeze(0),
                "orig_attention_mask": enc_orig["attention_mask"].squeeze(0),
                "trans_input_ids": enc_trans["input_ids"].squeeze(0),
                "trans_attention_mask": enc_trans["attention_mask"].squeeze(0),
                "label": torch.tensor(label, dtype=torch.long)
            }

In [ ]:
# ============================================================
# 2. Model Definitions
# ============================================================

# ---- A. Concatenation Model ----
class ConcatModel(nn.Module):
    def __init__(self, model_name, num_labels):
        super().__init__()
        self.encoder = AutoModel.from_pretrained(model_name)
        hidden = self.encoder.config.hidden_size
        self.fc = nn.Sequential(
            nn.Linear(2*hidden, hidden),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden, num_labels)
        )

    def forward(self, orig_input_ids, orig_attention_mask, trans_input_ids, trans_attention_mask):
        orig_emb = self.encoder(input_ids=orig_input_ids, attention_mask=orig_attention_mask).last_hidden_state[:,0,:]
        trans_emb = self.encoder(input_ids=trans_input_ids, attention_mask=trans_attention_mask).last_hidden_state[:,0,:]
        combined = torch.cat([orig_emb, trans_emb], dim=-1)
        return self.fc(combined)


# ---- B. Difference Vector Model ----
class DiffModel(nn.Module):
    def __init__(self, model_name, num_labels):
        super().__init__()
        self.encoder = AutoModel.from_pretrained(model_name)
        hidden = self.encoder.config.hidden_size
        self.fc = nn.Sequential(
            nn.Linear(hidden, hidden),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden, num_labels)
        )

    def forward(self, orig_input_ids, orig_attention_mask, trans_input_ids, trans_attention_mask):
        orig_emb = self.encoder(input_ids=orig_input_ids, attention_mask=orig_attention_mask).last_hidden_state[:,0,:]
        trans_emb = self.encoder(input_ids=trans_input_ids, attention_mask=trans_attention_mask).last_hidden_state[:,0,:]
        diff = trans_emb - orig_emb
        return self.fc(diff)


# ---- C. Cross-Encoder Model ----
def build_cross_encoder(model_name, num_labels):
    return AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

In [ ]:
# ============================================================
# 3. Training Utilities
# ============================================================

def train_one_epoch(model, dataloader, optimizer, device, loss_fn):
    model.train()
    total_loss = 0
    for batch in tqdm(dataloader, desc="Training"):
        optimizer.zero_grad()
        for k in batch:
            batch[k] = batch[k].to(device)
        labels = batch["label"]

        if isinstance(model, (ConcatModel, DiffModel)):
            logits = model(
                batch["orig_input_ids"], batch["orig_attention_mask"],
                batch["trans_input_ids"], batch["trans_attention_mask"]
            )
            loss = loss_fn(logits, labels)
        else:
            outputs = model(
                input_ids=batch["input_ids"],
                attention_mask=batch["attention_mask"],
                labels=labels
            )
            loss = outputs.loss

        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(dataloader)


@torch.no_grad()
def evaluate(model, dataloader, device, loss_fn, label_names, verbose=False):
    model.eval()
    total_loss = 0
    preds, trues = [], []
    for batch in tqdm(dataloader, desc="Evaluating"):
        for k in batch:
            batch[k] = batch[k].to(device)
        labels = batch["label"]

        if isinstance(model, (ConcatModel, DiffModel)):
            logits = model(
                batch["orig_input_ids"], batch["orig_attention_mask"],
                batch["trans_input_ids"], batch["trans_attention_mask"]
            )
            loss = loss_fn(logits, labels)
        else:
            outputs = model(
                input_ids=batch["input_ids"],
                attention_mask=batch["attention_mask"],
                labels=labels
            )
            logits = outputs.logits
            loss = outputs.loss

        total_loss += loss.item()
        pred = torch.argmax(logits, dim=-1)
        preds.extend(pred.cpu().tolist())
        trues.extend(labels.cpu().tolist())

    avg_loss = total_loss / len(dataloader)
    acc = accuracy_score(trues, preds)
    f1_macro = f1_score(trues, preds, average='macro')
    f1_weighted = f1_score(trues, preds, average='weighted')
    precision = precision_score(trues, preds, average='macro')
    recall = recall_score(trues, preds, average='macro')
    metrics = {"accuracy": acc, "f1_macro": f1_macro, "f1_weighted": f1_weighted, "precision": precision, "recall": recall}

    report_str = classification_report(trues, preds, target_names=label_names)
    if verbose:
        print("\nClassification Report:\n", report_str)

    return avg_loss, metrics, report_str

In [ ]:
def run_experiment(model_type='cross', model_name='roberta-base', epochs=3, batch_size=8, lr=2e-5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # ---- Load and preprocess data ----
    file_path = "/content/style_data.xlsx"
    try:
        df = pd.read_excel(file_path)
        print("Loaded Excel file successfully.")
    except Exception as e:
        raise ValueError(f"Could not read the Excel file: {e}")

    # Clean column names
    df.columns = df.columns.str.strip()
    df = df.rename(columns={'CHNAGED STYLE': 'CHANGED STYLE'})
    df.columns = df.columns.str.upper()

    print("Available columns after cleaning:", df.columns.tolist())

    # Check if required columns exist
    required_cols = ["ORIGINAL TRANSCRIPTS", "CHANGED STYLE", "PROMPT STYLE"]
    missing_cols = [col for col in required_cols if col not in df.columns]
    if missing_cols:
        raise KeyError(f"Missing required columns: {missing_cols}")

    df = df.dropna(subset=required_cols)
    for col in required_cols:
        df[col] = df[col].astype(str).str.strip()
    df = df[(df["ORIGINAL TRANSCRIPTS"] != "") & (df["CHANGED STYLE"] != "") & (df["PROMPT STYLE"] != "")]

    # Filter out classes with fewer than 2 samples for stratified split
    class_counts = df["PROMPT STYLE"].value_counts()
    valid_labels = class_counts[class_counts >= 2].index.tolist()
    df = df[df["PROMPT STYLE"].isin(valid_labels)]

    if len(df) == 0:
        raise ValueError("No valid data after filtering singletons.")

    labels = sorted(df["PROMPT STYLE"].unique())
    label2id = {y: i for i, y in enumerate(labels)}
    id2label = {i: y for i, y in enumerate(labels)}

    train_df, test_df = train_test_split(df, test_size=0.2, stratify=df["PROMPT STYLE"], random_state=42)
    val_df, test_df = train_test_split(test_df, test_size=0.5, stratify=test_df["PROMPT STYLE"], random_state=42)

    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

    train_ds = StyleDataset(train_df, tokenizer, label2id, model_type=model_type)
    val_ds = StyleDataset(val_df, tokenizer, label2id, model_type=model_type)
    test_ds = StyleDataset(test_df, tokenizer, label2id, model_type=model_type)

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=batch_size)
    test_loader = DataLoader(test_ds, batch_size=batch_size)

    # ---- Build Model ----
    if model_type == 'concat':
        model = ConcatModel(model_name, len(labels))
    elif model_type == 'diff':
        model = DiffModel(model_name, len(labels))
    else:
        model = build_cross_encoder(model_name, len(labels))

    model.to(device)

    optimizer = optim.AdamW(model.parameters(), lr=lr)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)
    loss_fn = nn.CrossEntropyLoss()

    # ---- Train ----
    print(f"\n--- Training {model_type.upper()} model ---")
    best_val_loss = float('inf')
    early_stop_patience = 3
    counter = 0
    for epoch in range(epochs):
        train_loss = train_one_epoch(model, train_loader, optimizer, device, loss_fn)
        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}")
        val_loss, val_metrics, _ = evaluate(model, val_loader, device, loss_fn, labels, verbose=False)
        print(f"Validation Loss: {val_loss:.4f}, Metrics: {val_metrics}")
        scheduler.step(val_loss)
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), f"{model_type}_best_model.pt")
            counter = 0
        else:
            counter += 1
        if counter >= early_stop_patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

    # Load best model for final evaluation
    model.load_state_dict(torch.load(f"{model_type}_best_model.pt"))

    # ---- Final Test ----
    print("\n--- Final Test Evaluation ---")
    _, test_metrics, test_report = evaluate(model, test_loader, device, loss_fn, labels, verbose=True)
    with open(f"{model_type}_classification_report.txt", "w") as f:
        f.write(test_report)
    torch.save(model.state_dict(), f"{model_type}_model.pt")
    return test_metrics

In [ ]:
# ============================================================
# 5. Run All Models
# ============================================================

if __name__ == "__main__":
    for m in ['concat', 'diff', 'cross']:
        results = run_experiment(model_type=m)
        print(f"\n>>> {m.upper()} MODEL RESULTS:", results)

Loaded Excel file successfully.
Available columns after cleaning: ['TOPIC NAME', 'YOUTUDE LINK', 'TRANSCRIPTS', 'ORIGINAL TRANSCRIPTS', 'PROMPT STYLE', 'CHANGED STYLE', 'PROMPT']


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

Epoch 3/3, Train Loss: 0.8740


Evaluating:   0%|          | 0/5 [00:00<?, ?it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned f

Validation Loss: 0.8674, Metrics: {'accuracy': 0.717948717948718, 'f1_macro': 0.208955223880597, 'f1_weighted': 0.6000765403750479, 'precision': 0.1794871794871795, 'recall': 0.25}

--- Final Test Evaluation ---


Evaluating:   0%|          | 0/5 [00:00<?, ?it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned f


Classification Report:
               precision    recall  f1-score   support

      Formal       0.70      1.00      0.82        28
      formal       0.00      0.00      0.00         8
  optimistic       0.00      0.00      0.00         1
 pessimistic       0.00      0.00      0.00         3

    accuracy                           0.70        40
   macro avg       0.17      0.25      0.21        40
weighted avg       0.49      0.70      0.58        40


>>> CROSS MODEL RESULTS: {'accuracy': 0.7, 'f1_macro': 0.20588235294117646, 'f1_weighted': 0.5764705882352941, 'precision': 0.175, 'recall': 0.25}


In [ ]:
import pandas as pd

try:
    df_check = pd.read_excel("/content/style_data.xlsx")
    print("Column names in your DataFrame:")
    print(df_check.columns.tolist())
except FileNotFoundError:
    print("Error: style_data.xlsx not found. Please make sure the file is uploaded to /content/.")
except Exception as e:
    print(f"An error occurred while reading the Excel file: {e}")